# Konvolucija Toplicovim i cirkularnim matricama

- Matrica je **Toplicova (dijagonalno konstantna)** ukoliko su elementi na svakoj dijagonali koja je paralelna glavnoj jednaki, tj. svaka matrica oblika:

  $A = \begin{bmatrix}
a_0 & a_{-1} & a_{-2} & \cdots & \cdots & a_{-(n-1)} \\
a_1 & a_0 & a_{-1} & \ddots &  & \vdots \\
a_2 & a_1 & a_0 & \ddots & \ddots & \vdots \\
\vdots & \ddots & \ddots & \ddots & a_{-1} & a_{-2} \\
\vdots & & \ddots & a_1 & a_0 & a_{-1} \\
a_{n-1} & \cdots & \cdots & a_2 & a_1 & a_0
\end{bmatrix}$

  je Toplicova.
- Matrica je **Cirkularna** ako je oblika:

  $A=\begin{bmatrix}
c_0 & c_{n-1} & \cdots & c_2 & c_1 \\
c_1 & c_0 & c_{n-1} &  & c_2\\
\vdots & c_1 & c_0 & \ddots & \vdots \\
c_{n-2} & & \ddots & \ddots & c_{n-1} \\
c_{n-1} & c_{n-2} & \cdots & c_1 & c_0 
\end{bmatrix}$

- Hoćemo da odradimo 2D konvoluciju matrice $f=\begin{bmatrix}
2 & 5 & 3 \\
1 & 4 & 1
\end{bmatrix}$ kernelom $k=\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}$,   pri čemu su dimenzije matrica redom $(M1,N1)=(3,2)$ i $(M2,N2)=(2,2)$.     Dimenzija izlaza je tada $(M1+M2-1,N1+N2-1)=(4,3)$
- Prvo proširujemo h nulama, tako da joj dimenzija bude jednaka dimenziji izlaza: $h=\begin{bmatrix} 1 & 2 & 0 & 0 \\ 3 & 4 & 0 & 0 \\ 0 & 0 & 0 & 0  \end{bmatrix}$
- Za svaki red matrice h, konstruišemo matricu sa 3 kolone (broj kolona matrice f):
  $H1=\begin{bmatrix} 1 & 0 & 0 \\ 2 & 1 & 0 \\ 0 & 2 & 1 \\ 0 & 0 & 2  \end{bmatrix}
 H2=\begin{bmatrix} 3 & 0 & 0 \\ 4 & 3 & 0 \\ 0 & 4 & 3 \\ 0 & 0 & 4  \end{bmatrix}
 H3=\begin{bmatrix} 0 & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 0 \\ 0 & 0 & 0  \end{bmatrix}$
 
- Koristeći matrice $H1,H2,H3$ konstruišemo dvostruko Toplicovu matricu sa dve kolone (broj redova matrice f): $H=\begin{bmatrix} H1 & H3 \\ H2 & H1 \\ H3 & H2\end{bmatrix}$
- Konstruišemo vektor $\textbf{f}=\begin{bmatrix}2 \\ 5 \\ 3 \\ 1 \\ 4\\1\end{bmatrix}$ koristeći elemente matrice f. 
- Rezultat se dobija od vektora $g=H\textbf{f}$, kada ga transformišemo u matricu odgovarajuće dimenzije.

In [3]:
import numpy as np
from scipy import signal as sg
from time import time

In [4]:
#pomocna funkcija
def formHi(v,N1):
    R=np.array([v]).T
    
    for i in range(N1-1):
        v=np.insert(v,0,v[-1])
        v=np.delete(v,-1)
        R=np.hstack((R,np.array([v]).T))
    return R

#pomocna funkcija
def formH(Hlist,M1,N3):
    
    vertical=Hlist[0]
    for i in range(1,len(Hlist)):
        vertical=np.vstack((vertical,Hlist[i]))
    
    R=vertical.copy()
    
    for _ in range(M1-1):
        for i in range(N3):
            vertical=np.insert(vertical,0,vertical[-1-i],axis=0)
        for i in range(N3):
            vertical=np.delete(vertical,-1,axis=0)
        R=np.hstack((R,vertical))
    
    return R
#pomocna funkcija
def formF(A):
    R=np.array([])
    for row in A:
        R=np.append(R,row)
    return R.reshape(-1,1)
#glavna funkcija
def toeplitzConvolution(A,K):
    """
    Vraća 2D konvoluciju matrice A kernelom K, formirajući dvostruko Toplicovu matricu
    """
    M1,N1=A.shape
    M2,N2=K.shape
    M3=M1+M2-1
    N3=N1+N2-1
    
    K_padded=np.zeros((M3,N3))
    K_padded[:M2,:N2]=K
    
    f=formF(A)
    
    Hlist=[]
    for row in K_padded:
        Hlist.append(formHi(row,N1))
    
    H=formH(Hlist,M1,N3)
    
    return np.dot(H,f).reshape(M3,N3)

In [9]:
a=np.random.randint(0,100,(50,50))
b=np.random.randint(0,10,(3,3))
t0=time()
y=sg.convolve2d(a,b)
t1=time()
x=toeplitzConvolution(a,b)
t2=time()

#print(sum(x-y))
print('signal.convolve2d:',t1-t0)
print('toeplitz convolution:',-t1+t2)
print('---------------------------------')

a=np.random.randint(0,100,(50,50))
b=np.random.randint(0,10,(30,30))
t0=time()
y=sg.convolve2d(a,b)
t1=time()
x=toeplitzConvolution(a,b)
t2=time()

#print(sum(x-y))
print('signal.convolve2d:',t1-t0)
print('toeplitz convolution:',-t1+t2)


signal.convolve2d: 0.0005261898040771484
toeplitz convolution: 1.7817537784576416
---------------------------------
signal.convolve2d: 0.013386964797973633
toeplitz convolution: 6.537777423858643


+ Vidimo da kada povećamo dimenziju kernela, vreme izvršavanja bibliotečke konvolucije se poveca znatno više nego vreme izvšavanje konvolucije pomoću Toplicove matrice.
- Razlog predugog izvršavanja napisane funkcije je to što numpy biblioteka ne radi inplace pomeranje vektora, već i za samo dodavanje i skidanje sa kraja, alocira novi memorijski blok i kopira elemente.

In [10]:
def circularConvolution(v,k):
    """
    Formira cirkularnu matricu C od vektora v i vraća konvoluciju vektora v sa kernelom k, 
    zajedno sa matricom C.
    """
    C=np.array([])
    C=v.reshape(-1,1)
    for i in range(v.size-k.size):
        k=np.append(k,0)
        
    for i in range(v.size-1):
        v=np.insert(v,0,v[-1])
        v=np.delete(v,-1)
        C=np.hstack((C,np.array([v]).T))
    return np.dot(C,k),C

def circulantMatrixEig(C):
    """
    Vraća sopstvene vektore cirkularne matrice C kao kolone rezultujuće matrice.
    """
    n=C.shape[0]
    F=np.zeros((n,n),dtype='complex')
    
    for j in range(C.shape[0]):
        for k in range(C.shape[0]):
            F[j,k]=np.complex(np.cos(2*np.pi*j*k/n),np.sin(2*np.pi*j*k/n))
    return F

Za periodični ulaz x=[1 8 3 2 5] i odgovor sistema h=[3 5 2 4 1]:

In [17]:
x,C=circularConvolution(np.array([1,8,3,2,5]),np.array([3,5,2,4,1]))
print('x*h =',x)
print('C = \n',C)

x*h = [52 50 73 46 64]
C = 
 [[1 5 2 3 8]
 [8 1 5 2 3]
 [3 8 1 5 2]
 [2 3 8 1 5]
 [5 2 3 8 1]]


Za svaku cirkularnu matricu C, jedan sopstveni vektor je uvek $\begin{bmatrix}1 \\ 1 \\ \cdots \\ 1\end{bmatrix}$ i odgovarajuća sopstvena vrednost je suma svih elemenata u jednom (bilo kojem) od redova matrice C.

In [18]:
print('Eigen vector:',np.ones(C.shape[0]))
print('Corresponding eigen value:',np.sum(C[0]))

Eigen vector: [1. 1. 1. 1. 1.]
Corresponding eigen value: 19


Kako doći do svih sopstvenih vektora cirkularne matrice C? Primetimo da za $w_n=e^{\frac{2\pi i}{n}}$ važi $w_n^{j+n}=w_n^jw_n^n=w_n^j$, tj. povećavanjem eksponenata vrednosti se periodično smenjuju, baš kao i vrednosti $c_i$

<img src='slika1.jpg'>

U terminima $w_n$, k-ti sopstveni vektor $x^{(k)}$ proizvoljne $n\times n$ matrice je:
$x^{(k)}=\begin{bmatrix}w_n^{0k} \\ w_n^{1k}\\ \vdots \\ w_n^{(n-1)k}\end{bmatrix}$, pa je matrica sopstvenih vektora oblika:
$F=\begin{bmatrix}x^{(0)} & x^{(1)}&\cdots &x^{(n-1)}\end{bmatrix}$
Množenjem vektora ovom matricom dobija se baš diskretna furijeova transformacija vektora, pa se ova matrica naziva **Furijeova matrica.**

Kako doći do sopstvenih vrednosti matrice C? 
Pogledajmo proizvod $y=Cx^{(k)}$. Svaka komponenta se dobija na sledeći način:
$y_l=\sum_{j=0}^{n-1}{c_{j-l}w_n^{jk}}=w_n^{lk}\sum_{j=0}^{n-1}{c_{j-l}w_n^{(j-l)k}}$. <br>Međutim, pošto su $c_j$ i $w_n^j$ periodični, sve što preslikavanje indeksa $j \mapsto j-l$ radi jeste kružno pomeranje elemenata koji se sumiraju, tako da dobijamo istu sumu. <br>Sada na osnovu $Cx^{(k)}=\lambda_k x^{(k)}$ dobijamo $\lambda_k = \sum_{j=0}^{n-1}{c_jw_n^{jk}}$. U terminima matrice F i vektora c (prva vrsta matrice C) vektor sopstvenih vrednosti $c_{\lambda}$ dobijamo kao $c_{\lambda}=Fc$

Za cirkularne matrice C1 i C2 iste dimenzije, svi sopstveni vektori su jednaki:

In [19]:
x1,C1=circularConvolution(np.random.randint(0,100,4),np.array([1,2,3]))
x2,C2=circularConvolution(np.random.randint(0,100,4),np.array([1,2,3]))

vectors1=np.linalg.eig(C1)[1]
vectors2=np.linalg.eig(C2)[1]
eig_vectors_diff=[]
for i in range(vectors1.shape[1]):
    eig_vectors_diff.append(sum(vectors1[:,i]-vectors2[:,i]))
    
print(np.sum(eig_vectors_diff))

(4.000000000000002+5.33340727540123e-17j)


In [21]:
F=circulantMatrixEig(C1)
#normiranje kolona je potrebno naknadno odraditi
#for i in range(F.shape[0]):
#    F[:,i]=F[:,i]/np.linalg.norm(F[:,i])

Sopstveni vektori svih matrica dimenzije (4,4):

In [22]:
for i in range(F.shape[0]):
    print(F[:,i])

[1.+0.j 1.+0.j 1.+0.j 1.+0.j]
[ 1.0000000e+00+0.0000000e+00j  6.1232340e-17+1.0000000e+00j
 -1.0000000e+00+1.2246468e-16j -1.8369702e-16-1.0000000e+00j]
[ 1.+0.0000000e+00j -1.+1.2246468e-16j  1.-2.4492936e-16j
 -1.+3.6739404e-16j]
[ 1.0000000e+00+0.0000000e+00j -1.8369702e-16-1.0000000e+00j
 -1.0000000e+00+3.6739404e-16j  5.5109106e-16+1.0000000e+00j]


In [23]:
G=circulantMatrixEig(C2)
for i in range(G.shape[0]):
    G[:,i]=G[:,i]/np.linalg.norm(G[:,i])
    
for i in range(G.shape[0]):
    print(G[:,i])

[0.5+0.j 0.5+0.j 0.5+0.j 0.5+0.j]
[ 5.00000000e-01+0.000000e+00j  3.06161700e-17+5.000000e-01j
 -5.00000000e-01+6.123234e-17j -9.18485099e-17-5.000000e-01j]
[ 0.5+0.0000000e+00j -0.5+6.1232340e-17j  0.5-1.2246468e-16j
 -0.5+1.8369702e-16j]
[ 5.00000000e-01+0.0000000e+00j -9.18485099e-17-5.0000000e-01j
 -5.00000000e-01+1.8369702e-16j  2.75545530e-16+5.0000000e-01j]


Sopstvene vrednosti dobijamo množenjem matrice F prvim redom matrice C:

In [24]:
eigenvals=np.dot(F,C1[0,:])
eigenvals

array([184.+0.0000000e+00j, -50.+2.0000000e+01j,  16.-2.4492936e-16j,
       -50.-2.0000000e+01j])

In [25]:
np.linalg.eigvals(C1)

array([184. +0.j,  16. +0.j, -50.+20.j, -50.-20.j])